## Step 1: Install Required Packages

First, we need to install CrewAI and its dependencies. We'll use:
- `crewai`: The main framework
- `crewai-tools`: Pre-built tools for agents

In [1]:
# Install required packages
!pip install -q crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.4/814.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.9/763.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Setup API Keys and Environment

To use OpenAI's GPT-4.1-mini model, you need an API key and base URL.

**Security Best Practice:** Store your API key in a `.env` file, not in the code!

In [2]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

Enter your OpenAI API Key: ··········
Enter your OpenAI Base URL: ··········


## Step 3: Create Custom Tools

Tools are functions that agents can use to perform specific tasks. Let's create custom tools for our movie production assistant.

### Tool 1: Script Sentiment Analyzer
Analyzes the emotional tone of a script excerpt.

In [3]:
from crewai.tools import tool
import re

@tool("Script Sentiment Analyzer")
def analyze_script_sentiment(script_text: str) -> str:
    """
    Analyzes the sentiment and emotional tone of a movie script excerpt.
    Returns sentiment score and dominant emotions.

    Args:
        script_text: A string containing the script excerpt to analyze
    """
    # Simple sentiment analysis based on keywords
    positive_words = ['love', 'happy', 'joy', 'wonderful', 'amazing', 'brilliant', 'triumph', 'success']
    negative_words = ['hate', 'sad', 'angry', 'terrible', 'awful', 'tragedy', 'failure', 'death']
    suspense_words = ['mystery', 'unknown', 'hidden', 'secret', 'suspense', 'thriller']

    script_lower = script_text.lower()

    pos_count = sum(1 for word in positive_words if word in script_lower)
    neg_count = sum(1 for word in negative_words if word in script_lower)
    sus_count = sum(1 for word in suspense_words if word in script_lower)

    total = pos_count + neg_count + sus_count

    if total == 0:
        return "Sentiment: Neutral - No strong emotional indicators detected."

    result = f"Sentiment Analysis Results:\n"
    result += f"- Positive tone: {pos_count}/{total} ({pos_count/total*100:.1f}%)\n"
    result += f"- Negative tone: {neg_count}/{total} ({neg_count/total*100:.1f}%)\n"
    result += f"- Suspenseful tone: {sus_count}/{total} ({sus_count/total*100:.1f}%)\n"

    if pos_count > neg_count and pos_count > sus_count:
        result += "\nOverall: Uplifting and positive story"
    elif neg_count > pos_count and neg_count > sus_count:
        result += "\nOverall: Dark and dramatic narrative"
    else:
        result += "\nOverall: Tense and mysterious atmosphere"

    return result

# Test the tool
test_script = "A tale of love and triumph as Sarah discovers the hidden mystery that brings her joy."
print(analyze_script_sentiment.run(test_script))

Sentiment Analysis Results:
- Positive tone: 3/5 (60.0%)
- Negative tone: 0/5 (0.0%)
- Suspenseful tone: 2/5 (40.0%)

Overall: Uplifting and positive story


### Tool 2: Budget Calculator
Calculates production costs based on different categories.

In [4]:
@tool("Production Budget Calculator")
def calculate_production_budget(category: str, days: int = 30, crew_size: int = 50) -> str:
    """
    Calculates estimated production budget for different movie categories.

    Args:
        category: Type of production (indie, medium, blockbuster)
        days: Number of shooting days
        crew_size: Number of crew members
    """
    # Base daily rates per category
    rates = {
        'indie': {'daily': 50000, 'crew': 500, 'equipment': 10000},
        'medium': {'daily': 200000, 'crew': 2000, 'equipment': 50000},
        'blockbuster': {'daily': 1000000, 'crew': 10000, 'equipment': 250000}
    }

    category = category.lower()
    if category not in rates:
        return f"Unknown category. Choose from: {', '.join(rates.keys())}"

    rate = rates[category]

    production_cost = rate['daily'] * days
    crew_cost = rate['crew'] * crew_size * days
    equipment_cost = rate['equipment'] * days
    contingency = (production_cost + crew_cost + equipment_cost) * 0.15

    total = production_cost + crew_cost + equipment_cost + contingency

    result = f"📊 Budget Breakdown for {category.upper()} Production:\n"
    result += f"\n🎬 Production Costs: ${production_cost:,}"
    result += f"\n👥 Crew Costs ({crew_size} members): ${crew_cost:,}"
    result += f"\n📹 Equipment Rental: ${equipment_cost:,}"
    result += f"\n🛡️ Contingency (15%): ${contingency:,}"
    result += f"\n\n💰 TOTAL ESTIMATED BUDGET: ${total:,}"
    result += f"\n📅 For {days} shooting days"

    return result

# Test the tool
print(calculate_production_budget.run("medium", days=45, crew_size=75))

📊 Budget Breakdown for MEDIUM Production:

🎬 Production Costs: $9,000,000
👥 Crew Costs (75 members): $6,750,000
📹 Equipment Rental: $2,250,000
🛡️ Contingency (15%): $2,700,000.0

💰 TOTAL ESTIMATED BUDGET: $20,700,000.0
📅 For 45 shooting days


### Tool 3: Soundtrack Genre Recommender
Suggests music genres based on the movie's theme and mood.

In [5]:
@tool("Soundtrack Genre Recommender")
def recommend_soundtrack_genre(movie_genre: str, mood: str) -> str:
    """
    Recommends soundtrack genres based on movie genre and mood.

    Args:
        movie_genre: The genre of the movie (action, drama, comedy, horror, sci-fi, romance)
        mood: The desired mood (intense, light, emotional, mysterious, upbeat)
    """
    recommendations = {
        'action': {
            'intense': ['Epic Orchestral', 'Electronic/Synth', 'Heavy Metal'],
            'light': ['Pop Rock', 'Funk', 'Electronic Pop'],
            'emotional': ['Cinematic Strings', 'Piano & Strings', 'Alternative Rock'],
        },
        'drama': {
            'intense': ['Classical', 'Dramatic Orchestral', 'Jazz Noir'],
            'emotional': ['Piano Solo', 'String Quartet', 'Acoustic Guitar'],
            'mysterious': ['Ambient', 'Minimalist Piano', 'Contemporary Classical'],
        },
        'comedy': {
            'upbeat': ['Jazz', 'Pop', 'Ska', 'Funk'],
            'light': ['Acoustic Pop', 'Ukulele', 'Whistling & Bells'],
        },
        'horror': {
            'intense': ['Dark Ambient', 'Industrial', 'Atonal Orchestral'],
            'mysterious': ['Theremin', 'Dissonant Strings', 'Electronic Drone'],
        },
        'sci-fi': {
            'intense': ['Electronic/Synth', 'Cyberpunk', 'Orchestral Hybrid'],
            'mysterious': ['Ambient Electronic', 'Experimental', 'Synth Pad'],
        },
        'romance': {
            'emotional': ['Classical Piano', 'Acoustic', 'Indie Folk'],
            'light': ['Soft Pop', 'Bossa Nova', 'Acoustic Guitar'],
            'upbeat': ['Pop', 'Soul', 'Light Jazz'],
        }
    }

    genre = movie_genre.lower()
    mood_key = mood.lower()

    if genre not in recommendations:
        return f"Unknown genre. Try: {', '.join(recommendations.keys())}"

    genre_moods = recommendations[genre]
    if mood_key not in genre_moods:
        return f"For {genre}, available moods are: {', '.join(genre_moods.keys())}"

    suggested = genre_moods[mood_key]

    result = f"🎵 Soundtrack Recommendations for {genre.upper()} ({mood} mood):\n\n"
    for i, genre_rec in enumerate(suggested, 1):
        result += f"{i}. {genre_rec}\n"

    result += f"\n💡 Tip: Consider mixing {suggested[0]} with {suggested[-1]} for dynamic scenes!"

    return result

# Test the tool
print(recommend_soundtrack_genre.run("sci-fi", "mysterious"))

🎵 Soundtrack Recommendations for SCI-FI (mysterious mood):

1. Ambient Electronic
2. Experimental
3. Synth Pad

💡 Tip: Consider mixing Ambient Electronic with Synth Pad for dynamic scenes!


## Step 4: Integrate MCP Tool (Model Context Protocol)

MCP tools allow agents to interact with external systems and APIs. Here we'll create a simulated MCP tool for accessing a movie database.

**Note:** In production, MCP tools connect to real external services. This is a simplified demonstration.

In [6]:
from typing import Dict, Any

@tool("Movie Market Research MCP")
def fetch_market_data(genre: str) -> str:
    """
    Fetches market research data for a specific movie genre from the database.
    This simulates an MCP tool that connects to external market research APIs.

    Args:
        genre: The movie genre to research (action, drama, comedy, horror, sci-fi, romance)
    """
    # Simulated market data (in production, this would call a real API)
    market_database = {
        'action': {
            'avg_box_office': '$450M',
            'audience_demographic': '18-35 years, 65% male',
            'peak_season': 'Summer (May-August)',
            'trending_themes': ['Superheroes', 'International espionage', 'Disaster scenarios'],
            'streaming_performance': 'High - 85% completion rate',
        },
        'drama': {
            'avg_box_office': '$120M',
            'audience_demographic': '30-60 years, 55% female',
            'peak_season': 'Awards season (October-February)',
            'trending_themes': ['Social justice', 'Biography', 'Family dynamics'],
            'streaming_performance': 'Medium - 72% completion rate',
        },
        'comedy': {
            'avg_box_office': '$180M',
            'audience_demographic': '18-45 years, balanced gender',
            'peak_season': 'Year-round, slight spike in holidays',
            'trending_themes': ['Romantic comedy', 'Workplace humor', 'Cultural clash'],
            'streaming_performance': 'Very High - 90% completion rate',
        },
        'horror': {
            'avg_box_office': '$95M',
            'audience_demographic': '16-30 years, 52% male',
            'peak_season': 'Fall (September-November)',
            'trending_themes': ['Psychological horror', 'Folk horror', 'Home invasion'],
            'streaming_performance': 'High - 82% completion rate',
        },
        'sci-fi': {
            'avg_box_office': '$380M',
            'audience_demographic': '18-40 years, 60% male',
            'peak_season': 'Summer and Holiday season',
            'trending_themes': ['AI & technology', 'Space exploration', 'Time travel'],
            'streaming_performance': 'High - 88% completion rate',
        },
        'romance': {
            'avg_box_office': '$85M',
            'audience_demographic': '18-50 years, 70% female',
            'peak_season': 'Valentine\'s Day season (January-February)',
            'trending_themes': ['Second chance romance', 'Holiday romance', 'Cross-cultural love'],
            'streaming_performance': 'Very High - 92% completion rate',
        },
    }

    genre_key = genre.lower()

    if genre_key not in market_database:
        return f"❌ No market data available for '{genre}'. Available genres: {', '.join(market_database.keys())}"

    data = market_database[genre_key]

    result = f"📈 Market Research Report: {genre.upper()} Genre\n"
    result += f"\n💰 Average Box Office: {data['avg_box_office']}"
    result += f"\n👥 Target Demographic: {data['audience_demographic']}"
    result += f"\n📅 Best Release Window: {data['peak_season']}"
    result += f"\n🎬 Trending Themes:"
    for theme in data['trending_themes']:
        result += f"\n   • {theme}"
    result += f"\n📺 Streaming Performance: {data['streaming_performance']}"
    result += f"\n\n✅ Data retrieved from Market Intelligence Database (MCP)"

    return result

# Test the MCP tool
print(fetch_market_data.run("action"))

📈 Market Research Report: ACTION Genre

💰 Average Box Office: $450M
👥 Target Demographic: 18-35 years, 65% male
📅 Best Release Window: Summer (May-August)
🎬 Trending Themes:
   • Superheroes
   • International espionage
   • Disaster scenarios
📺 Streaming Performance: High - 85% completion rate

✅ Data retrieved from Market Intelligence Database (MCP)


## Step 5: Configure the Language Model

Now let's set up OpenAI's GPT-4.1-mini model to power our agents. This model is fast and cost-effective, perfect for agent-based workflows.

In [7]:
from crewai import LLM

# Initialize GPT-4.1-mini using CrewAI's LLM wrapper
llm = LLM(
    model="openai/gpt-4.1-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ["OPENAI_BASE_URL"],
    temperature=0.7,  # Controls creativity (0=focused, 1=creative)
)

print("✅ Language model configured: GPT-4.1-mini")
print("   - Model: openai/gpt-4.1-mini")
print("   - Temperature: 0.7 (balanced creativity)")

✅ Language model configured: GPT-4.1-mini
   - Model: openai/gpt-4.1-mini
   - Temperature: 0.7 (balanced creativity)


## Step 6: Create Specialized Agents

Agents are AI assistants with specific roles and tools. Let's create a crew of specialized agents for movie production:

1. **Script Analyst** - Analyzes scripts and provides creative feedback
2. **Budget Manager** - Handles financial planning and budgeting
3. **Market Researcher** - Provides market insights and recommendations

In [8]:
from crewai import Agent

# Agent 1: Script Analyst
script_analyst = Agent(
    role="Script Analyst",
    goal="Analyze movie scripts for sentiment, themes, and recommend appropriate soundtracks",
    backstory="""You are an experienced script analyst with 15 years in Hollywood.
    You have a keen eye for emotional depth and understand how music enhances storytelling.
    You've worked on numerous award-winning films and know what makes a script compelling.""",
    tools=[analyze_script_sentiment, recommend_soundtrack_genre],
    llm=llm,
    verbose=True,  # Shows detailed thinking process
    allow_delegation=False,  # Can't delegate tasks to other agents
)

# Agent 2: Budget Manager
budget_manager = Agent(
    role="Production Budget Manager",
    goal="Calculate accurate production budgets and optimize spending across departments",
    backstory="""You are a meticulous financial expert specializing in film production.
    With an MBA and 10 years of experience managing budgets for indie to blockbuster films,
    you ensure projects stay financially viable while maintaining creative vision.""",
    tools=[calculate_production_budget],
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

# Agent 3: Market Researcher
market_researcher = Agent(
    role="Film Market Research Analyst",
    goal="Provide data-driven insights about market trends, audience preferences, and optimal release strategies",
    backstory="""You are a data scientist specializing in entertainment industry analytics.
    You analyze box office trends, streaming data, and audience demographics to help studios
    make informed decisions about production and distribution.""",
    tools=[fetch_market_data],
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

print("✅ Created 3 specialized agents:")
print("   1. Script Analyst (sentiment & soundtrack)")
print("   2. Budget Manager (financial planning)")
print("   3. Market Researcher (market intelligence)")

✅ Created 3 specialized agents:
   1. Script Analyst (sentiment & soundtrack)
   2. Budget Manager (financial planning)
   3. Market Researcher (market intelligence)


## Step 7: Define Tasks for Agents

Tasks are specific assignments given to agents. Each task has:
- A clear description
- An assigned agent
- Expected output format

In [9]:
from crewai import Task

# Sample movie concept for our tasks
movie_concept = """
Title: "Echoes of Tomorrow"
Genre: Sci-Fi Thriller
Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals
humanity's tragic future. Racing against time and a mysterious organization, she must decide
whether to alter the timeline or accept humanity's fate.

Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both
triumph and terror. The hidden secret she uncovers could save millions, but the mystery
of who left this message haunts her every waking moment."
"""

# Task 1: Analyze the script
task_analyze_script = Task(
    description=f"""Analyze the following movie concept and script excerpt:

    {movie_concept}

    Use your tools to:
    1. Analyze the sentiment of the script excerpt
    2. Recommend appropriate soundtrack genres based on the genre and mood
    3. Provide creative suggestions for enhancing the emotional impact
    """,
    expected_output="A detailed analysis including sentiment breakdown, soundtrack recommendations, and creative suggestions",
    agent=script_analyst,
)

# Task 2: Calculate budget
task_budget_calculation = Task(
    description="""Based on the sci-fi thriller movie concept, calculate the production budget.

    Assume:
    - This is a MEDIUM budget production
    - 60 days of shooting
    - Crew size of 85 people

    Provide a detailed breakdown and recommendations for budget optimization.
    """,
    expected_output="Complete budget breakdown with optimization recommendations",
    agent=budget_manager,
)

# Task 3: Market research
task_market_research = Task(
    description="""Conduct market research for the sci-fi thriller genre.

    Use the MCP tool to fetch market data and provide:
    1. Current market performance of sci-fi films
    2. Target audience insights
    3. Optimal release timing
    4. Strategic recommendations based on trending themes
    """,
    expected_output="Comprehensive market analysis with strategic release recommendations",
    agent=market_researcher,
)

print("✅ Created 3 tasks:")
print("   1. Script Analysis Task")
print("   2. Budget Calculation Task")
print("   3. Market Research Task")

✅ Created 3 tasks:
   1. Script Analysis Task
   2. Budget Calculation Task
   3. Market Research Task


## Step 8: Assemble the Crew and Execute

Now we bring it all together! A Crew coordinates multiple agents working on related tasks.

**Process Types:**
- `sequential`: Tasks execute one after another (used here)
- `hierarchical`: A manager agent coordinates other agents

In [10]:
from crewai import Crew, Process

# Assemble the crew
movie_production_crew = Crew(
    agents=[script_analyst, budget_manager, market_researcher],
    tasks=[task_analyze_script, task_budget_calculation, task_market_research],
    process=Process.sequential,  # Tasks run in order
    verbose=True,  # Enable detailed output for learning
)

print("✅ Crew assembled with 3 agents and 3 tasks")
print("\n🚀 Starting production analysis...\n")
print("=" * 80)

# Execute the crew's tasks
result = movie_production_crew.kickoff()

print("\n" + "=" * 80)
print("\n✅ Analysis Complete!\n")

✅ Crew assembled with 3 agents and 3 tasks

🚀 Starting production analysis...



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  2982e005-85da-4a13-a883-be04d8a3623c                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Analyze the following movie concept and script excerpt:                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: "Echoes of Tomorrow"                                                                                    │
│  Genre: Sci-Fi Thriller                                                                                         │
│  Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals                     │
│  humanity's tragic future. Racing against time and a mysterious organization, she must decide                   │
│  whether to alter the timeline or accept humanity's fate.                                                       │
│                                                                                                                 │
│  Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both                        │
│  triumph and terror. The hidden secret she uncovers could save millions, but the mystery                        │
│  of who left this message haunts her every waking moment."                                                      │
│                                                                                                                 │
│                                                                                                                 │
│      Use your tools to:                                                                                         │
│      1. Analyze the sentiment of the script excerpt                                                             │
│      2. Recommend appropriate soundtrack genres based on the genre and mood                                     │
│      3. Provide creative suggestions for enhancing the emotional impact                                         │
│                                                                                                                 │
│  ID: 20081780-ed7c-4661-af3d-17f2b2214629                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze the following movie concept and script excerpt:                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: "Echoes of Tomorrow"                                                                                    │
│  Genre: Sci-Fi Thriller                                                                                         │
│  Logline: A brilliant AI researcher discovers a hidden message in quantum data that reveals                     │
│  humanity's tragic future. Racing against time and a mysterious organization, she must decide                   │
│  whether to alter the timeline or accept humanity's fate.                                                       │
│                                                                                                                 │
│  Script Excerpt: "In the depths of the unknown laboratory, Sarah's discovery brings both                        │
│  triumph and terror. The hidden secret she uncovers could save millions, but the mystery                        │
│  of who left this message haunts her every waking moment."                                                      │
│                                                                                                                 │
│                                                                                                                 │
│      Use your tools to:                                                                                         │
│      1. Analyze the sentiment of the script excerpt                                                             │
│      2. Recommend appropriate soundtrack genres based on the genre and mood                                     │
│      3. Provide creative suggestions for enhancing the emotional impact                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: script_sentiment_analyzer                                                                                │
│  Args: {'script_text': "In the depths of the unknown laboratory, Sarah's discovery brings both triumph and      │
│  terror. The hidden secret she uncovers could save millions, but the mystery of who left this message...        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────── ✅ Tool Execution Completed (#1) ────────────────────────────────────────╮
│                                                                                                                 │
│  Tool Completed                                                                                                 │
│  Tool: movie_market_research_mcp                                                                                │
│  Output: 📈 Market Research Report: SCI-FI Genre                                                                │
│                                                                                                                 │
│  💰 Average Box Office: $380M                                                                                   │
│  👥 Target Demographic: 18-40 years, 60% male                                                                   │
│  📅 Best Release Window: Summer and Holiday season                                                              │
│  🎬 Trending Themes:                                                                                            │
│     • AI & technology                                                                                           │
│     • Space exploration                                                                                         │
│     • Time travel                                                                                               │
│  📺 Streaming Performance: High - 88% completion rate                                                           │
│                                                                                                                 │
│  ✅ Data retrieved from Market Intelligence Database (MCP)                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Film Market Research Analyst                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Comprehensive Market Analysis for Sci-Fi Thriller Genre:                                                       │
│                                                                                                                 │
│  1. Current Market Performance of Sci-Fi Films:                                                                 │
│  Sci-fi films are performing strongly in the market with an average box office revenue of $380 million. The     │
│  genre demonstrates robust streaming performance with an 88% completion rate, indicating high audience          │
│  engagement and retention.                                                                                      │
│                                                                                                                 │
│  2. Target Audience Insights:                                                                                   │
│  The primary demographic for sci-fi films is individuals aged 18-40 years, with a predominant male audience     │
│  making up 60%. This suggests marketing efforts should be tailored to appeal to young adults, particularly      │
│  males who are drawn to futuristic and technology-driven narratives.                                            │
│                                                                                                                 │
│  3. Optimal Release Timing:                                                                                     │
│  The best release windows for sci-fi films are during the summer and holiday seasons. These periods align with  │
│  higher audience availability and increased entertainment consumption, maximizing box office potential.         │
│                                                                                                                 │
│  4. Strategic Recommendations Based on Trending Themes:                                                         │
│  Current trends in sci-fi revolve around themes of artificial intelligence and technology, space exploration,   │
│  and time travel. To capitalize on market interest, incorporating these elements into sci-fi thrillers can      │
│  enhance appeal and relevance.                                                                                  │
│                                                                                                                 │
│  Given the context of "Echoes of Tomorrow," which features a tense, mysterious atmosphere with themes of        │
│  hidden messages and suspense, leveraging AI and technology elements aligns well with market trends. The        │
│  soundtrack and emotional tone recommendations complement this, creating a compelling viewer experience.        │
│                                                                                                                 │
│  Strategically, releasing the film in summer or the holiday season will optimize visibility and audience        │
│  reach. Marketing should target the 18-40 demographic with emphasis on the sci-fi thriller's unique emotional   │
│  depth and futuristic themes.                                                                                   │
│                                                                                                                 │
│  Overall, the sci-fi thriller genre remains lucrative w

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Conduct market research for the sci-fi thriller genre.                                                         │
│                                                                                                                 │
│      Use the MCP tool to fetch market data and provide:                                                         │
│      1. Current market performance of sci-fi films                                                              │
│      2. Target audience insights                                                                                │
│      3. Optimal release timing                                                                                  │
│      4. Strategic recommendations based on trending themes                                                      │
│                                                                                                                 │
│  Agent:                                                                                                         │
│  Film Market Research Analyst                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



✅ Analysis Complete!



╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  2982e005-85da-4a13-a883-be04d8a3623c                                                                           │
│  Final Output: Comprehensive Market Analysis for Sci-Fi Thriller Genre:                                         │
│                                                                                                                 │
│  1. Current Market Performance of Sci-Fi Films:                                                                 │
│  Sci-fi films are performing strongly in the market with an average box office revenue of $380 million. The     │
│  genre demonstrates robust streaming performance with an 88% completion rate, indicating high audience          │
│  engagement and retention.                                                                                      │
│                                                                                                                 │
│  2. Target Audience Insights:                                                                                   │
│  The primary demographic for sci-fi films is individuals aged 18-40 years, with a predominant male audience     │
│  making up 60%. This suggests marketing efforts should be tailored to appeal to young adults, particularly      │
│  males who are drawn to futuristic and technology-driven narratives.                                            │
│                                                                                                                 │
│  3. Optimal Release Timing:                                                                                     │
│  The best release windows for sci-fi films are during the summer and holiday seasons. These periods align with  │
│  higher audience availability and increased entertainment consumption, maximizing box office potential.         │
│                                                                                                                 │
│  4. Strategic Recommendations Based on Trending Themes:                                                         │
│  Current trends in sci-fi revolve around themes of artificial intelligence and technology, space exploration,   │
│  and time travel. To capitalize on market interest, incorporating these elements into sci-fi thrillers can      │
│  enhance appeal and relevance.                                                                                  │
│                                                                                                                 │
│  Given the context of "Echoes of Tomorrow," which features a tense, mysterious atmosphere with themes of        │
│  hidden messages and suspense, leveraging AI and technology elements aligns well with market trends. The        │
│  soundtrack and emotional tone recommendations complement this, creating a compelling viewer experience.        │
│                                                                                                                 │
│  Strategically, releasing the film in summer or the holiday season will optimize visibility and audience        │
│  reach. Marketing should target the 18-40 demographic with emphasis on the sci-fi thriller's unique emotional   │
│  depth and futuristic themes.                         

## Step 9: View Final Results

Let's examine the consolidated output from all agents.

In [11]:
print("📋 FINAL PRODUCTION REPORT")
print("=" * 80)
print(result)
print("=" * 80)

📋 FINAL PRODUCTION REPORT
Comprehensive Market Analysis for Sci-Fi Thriller Genre:

1. Current Market Performance of Sci-Fi Films:
Sci-fi films are performing strongly in the market with an average box office revenue of $380 million. The genre demonstrates robust streaming performance with an 88% completion rate, indicating high audience engagement and retention.

2. Target Audience Insights:
The primary demographic for sci-fi films is individuals aged 18-40 years, with a predominant male audience making up 60%. This suggests marketing efforts should be tailored to appeal to young adults, particularly males who are drawn to futuristic and technology-driven narratives.

3. Optimal Release Timing:
The best release windows for sci-fi films are during the summer and holiday seasons. These periods align with higher audience availability and increased entertainment consumption, maximizing box office potential.

4. Strategic Recommendations Based on Trending Themes:
Current trends in sci-fi r

## Step 10: Implementing Observability

Observability helps you monitor and debug your agents. Let's add logging and tracking capabilities.

**Key Observability Features:**
1. Execution time tracking
2. Tool usage monitoring
3. Error logging
4. Agent decision tracking

In [12]:
import time
import json
from datetime import datetime

class CrewObserver:
    """Custom observability class for monitoring CrewAI execution"""

    def __init__(self):
        self.logs = []
        self.start_time = None
        self.end_time = None

    def log_event(self, event_type: str, agent_name: str, details: dict):
        """Log an event during crew execution"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'event_type': event_type,
            'agent': agent_name,
            'details': details
        }
        self.logs.append(log_entry)

    def start_monitoring(self):
        """Start monitoring crew execution"""
        self.start_time = time.time()
        self.log_event('CREW_START', 'System', {'message': 'Crew execution started'})

    def end_monitoring(self):
        """End monitoring and calculate metrics"""
        self.end_time = time.time()
        duration = self.end_time - self.start_time
        self.log_event('CREW_END', 'System', {
            'message': 'Crew execution completed',
            'duration_seconds': round(duration, 2)
        })

    def get_metrics(self):
        """Generate execution metrics"""
        if not self.end_time:
            return "Monitoring still in progress"

        duration = self.end_time - self.start_time
        agent_events = {}

        for log in self.logs:
            agent = log['agent']
            if agent != 'System':
                agent_events[agent] = agent_events.get(agent, 0) + 1

        metrics = f"""\n📊 EXECUTION METRICS\n{'='*60}
⏱️  Total Duration: {duration:.2f} seconds
📝 Total Events Logged: {len(self.logs)}
🤖 Agent Activity:"""

        for agent, count in agent_events.items():
            metrics += f"\n   • {agent}: {count} events"

        return metrics

    def display_logs(self, filter_agent=None):
        """Display execution logs"""
        print(f"\n📋 EXECUTION LOGS ({len(self.logs)} entries)")
        print("=" * 80)

        for log in self.logs:
            if filter_agent and log['agent'] != filter_agent:
                continue

            timestamp = log['timestamp'].split('T')[1].split('.')[0]
            print(f"[{timestamp}] {log['event_type']:15} | {log['agent']:25} | {log['details']}")

# Initialize observer
observer = CrewObserver()
print("✅ Observability system initialized")

✅ Observability system initialized


## Step 11: Run Crew with Observability

Now let's run our crew again with full observability enabled.

In [13]:
# Create a new task with observability hooks
observer.start_monitoring()

# Log agent initialization
observer.log_event('AGENT_INIT', 'Script Analyst', {'tools': ['sentiment_analyzer', 'soundtrack_recommender']})
observer.log_event('AGENT_INIT', 'Budget Manager', {'tools': ['budget_calculator']})
observer.log_event('AGENT_INIT', 'Market Researcher', {'tools': ['market_data_mcp']})

# New movie concept for second run
new_concept = """
Title: "The Last Symphony"
Genre: Drama
Logline: An elderly composer, losing her memory to dementia, races to complete her
final masterpiece while her estranged daughter learns to understand her through music.

Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across
the piano keys. The wonderful melody emerges from the fog of her fading memory, a
triumph over the terrible disease that threatens to steal her life's work."
"""

# Create new task
observed_task = Task(
    description=f"""Analyze this movie concept:
    {new_concept}

    Perform sentiment analysis and recommend soundtrack genres.""",
    expected_output="Sentiment analysis and soundtrack recommendations",
    agent=script_analyst,
)

# Log task start
observer.log_event('TASK_START', 'Script Analyst', {'task': 'Script analysis'})

# Create a smaller crew for demonstration
observed_crew = Crew(
    agents=[script_analyst],
    tasks=[observed_task],
    process=Process.sequential,
    verbose=1,  # Less verbose for cleaner output
)

print("🔍 Running crew with observability enabled...\n")

# Execute
result_observed = observed_crew.kickoff()

# Log completion
observer.log_event('TASK_COMPLETE', 'Script Analyst', {'status': 'success'})
observer.end_monitoring()

print("\n✅ Execution complete with observability tracking!")

🔍 Running crew with observability enabled...



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  3f9a8c37-7c23-4400-931d-ade34fc393da                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Analyze this movie concept:                                                                              │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│  ID: 730d7029-12e3-4d19-8170-c5439960f721                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│  Task: Analyze this movie concept:                                                                              │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Tool script_sentiment_analyzer executed with result: Sentiment Analysis Results:
- Positive tone: 3/5 (60.0%)
- Negative tone: 2/5 (40.0%)
- Suspenseful tone: 0/5 (0.0%)

Overall: Uplifting and positive story...

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Script Analyst                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Sentiment Analysis:                                                                                            │
│  The script excerpt for "The Last Symphony" is predominantly uplifting and positive with a 60% positive tone    │
│  and 40% negative tone. It beautifully captures the bittersweet emotions of joy and sadness intertwined as      │
│  Margaret, the elderly composer, plays her final masterpiece. The narrative highlights a triumphant struggle    │
│  against the ravages of dementia, emphasizing hope and resilience.                                              │
│                                                                                                                 │
│  Soundtrack Recommendations:                                                                                    │
│  For this emotional drama, the soundtrack should enhance the heartfelt and intimate nature of the story.        │
│  Recommended genres include:                                                                                    │
│  1. Piano Solo – to reflect the central role of music and Margaret's personal connection to the piano.          │
│  2. String Quartet – to add depth, warmth, and emotional resonance.                                             │
│  3. Acoustic Guitar – to bring an intimate, human touch that complements the piano and strings.                 │
│                                                                                                                 │
│  A blend of Piano Solo with Acoustic Guitar is suggested to create dynamic scenes that evolve with the story’s  │
│  emotional arc, highlighting both moments of introspection and connection between characters.                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Analyze this movie concept:                                                                                    │
│                                                                                                                 │
│  Title: "The Last Symphony"                                                                                     │
│  Genre: Drama                                                                                                   │
│  Logline: An elderly composer, losing her memory to dementia, races to complete her                             │
│  final masterpiece while her estranged daughter learns to understand her through music.                         │
│                                                                                                                 │
│  Script Excerpt: "Each note brings both joy and sadness as Margaret's fingers dance across                      │
│  the piano keys. The wonderful melody emerges from the fog of her fading memory, a                              │
│  triumph over the terrible disease that threatens to steal her life's work."                                    │
│                                                                                                                 │
│                                                                                                                 │
│      Perform sentiment analysis and recommend soundtrack genres.                                                │
│  Agent:                                                                                                         │
│  Script Analyst                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  3f9a8c37-7c23-4400-931d-ade34fc393da                                                                           │
│  Final Output: Sentiment Analysis:                                                                              │
│  The script excerpt for "The Last Symphony" is predominantly uplifting and positive with a 60% positive tone    │
│  and 40% negative tone. It beautifully captures the bittersweet emotions of joy and sadness intertwined as      │
│  Margaret, the elderly composer, plays her final masterpiece. The narrative highlights a triumphant struggle    │
│  against the ravages of dementia, emphasizing hope and resilience.                                              │
│                                                                                                                 │
│  Soundtrack Recommendations:                                                                                    │
│  For this emotional drama, the soundtrack should enhance the heartfelt and intimate nature of the story.        │
│  Recommended genres include:                                                                                    │
│  1. Piano Solo – to reflect the central role of music and Margaret's personal connection to the piano.          │
│  2. String Quartet – to add depth, warmth, and emotional resonance.                                             │
│  3. Acoustic Guitar – to bring an intimate, human touch that complements the piano and strings.                 │
│                                                                                                                 │
│  A blend of Piano Solo with Acoustic Guitar is suggested to create dynamic scenes that evolve with the story’s  │
│  emotional arc, highlighting both moments of introspection and connection between characters.                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Execution complete with observability tracking!


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 📈 Step 12: Analyze Observability Data

Let's examine the metrics and logs collected during execution.

In [14]:
# Display metrics
print(observer.get_metrics())

# Display all logs
observer.display_logs()

# Show result
print("\n📋 AGENT OUTPUT")
print("=" * 80)
print(result_observed)
print("=" * 80)


📊 EXECUTION METRICS
⏱️  Total Duration: 4.94 seconds
📝 Total Events Logged: 7
🤖 Agent Activity:
   • Script Analyst: 3 events
   • Budget Manager: 1 events
   • Market Researcher: 1 events

📋 EXECUTION LOGS (7 entries)
[11:28:49] CREW_START      | System                    | {'message': 'Crew execution started'}
[11:28:49] AGENT_INIT      | Script Analyst            | {'tools': ['sentiment_analyzer', 'soundtrack_recommender']}
[11:28:49] AGENT_INIT      | Budget Manager            | {'tools': ['budget_calculator']}
[11:28:49] AGENT_INIT      | Market Researcher         | {'tools': ['market_data_mcp']}
[11:28:49] TASK_START      | Script Analyst            | {'task': 'Script analysis'}
[11:28:54] TASK_COMPLETE   | Script Analyst            | {'status': 'success'}
[11:28:54] CREW_END        | System                    | {'message': 'Crew execution completed', 'duration_seconds': 4.94}

📋 AGENT OUTPUT
Sentiment Analysis:
The script excerpt for "The Last Symphony" is predominantly uplifti

## Step 13: Advanced Observability - Export Logs

In production, you'd want to export logs for analysis. Here's how to save them.

In [15]:
import json

# Export logs to JSON
def export_logs(observer, filename='crew_execution_logs.json'):
    """Export observability logs to a JSON file"""
    log_data = {
        'execution_summary': {
            'start_time': datetime.fromtimestamp(observer.start_time).isoformat(),
            'end_time': datetime.fromtimestamp(observer.end_time).isoformat(),
            'duration_seconds': round(observer.end_time - observer.start_time, 2),
            'total_events': len(observer.logs)
        },
        'events': observer.logs
    }

    with open(filename, 'w') as f:
        json.dump(log_data, f, indent=2)

    return f"✅ Logs exported to {filename}"

# Export the logs
print(export_logs(observer))

# Display a sample of the JSON structure
print("\n📄 Sample JSON structure:")
sample = {
    'execution_summary': 'metadata about the run',
    'events': [
        {
            'timestamp': '2025-11-30T10:30:45.123456',
            'event_type': 'TASK_START',
            'agent': 'Script Analyst',
            'details': {'task': 'Script analysis'}
        }
    ]
}
print(json.dumps(sample, indent=2))

✅ Logs exported to crew_execution_logs.json

📄 Sample JSON structure:
{
  "execution_summary": "metadata about the run",
  "events": [
    {
      "timestamp": "2025-11-30T10:30:45.123456",
      "event_type": "TASK_START",
      "agent": "Script Analyst",
      "details": {
        "task": "Script analysis"
      }
    }
  ]
}


## 🎯 CHALLENGE: Build Your Own Movie Production Crew!

Now it's your turn to apply what you've learned! Complete the following challenge:

### 🎬 Challenge Description

Create a **Movie Casting Assistant** crew that helps with casting decisions for a new film.

### Requirements:

1. **Create TWO custom tools:**
   - `actor_availability_checker`: Takes an actor name and date range, returns if they're available (simulate with logic)
   - `role_matcher`: Takes character description and returns suggested actor traits (age range, experience level, acting style)

2. **Create ONE MCP-style tool:**
   - `fetch_actor_database`: Simulates querying a database of actors with their filmography, awards, and availability

3. **Create TWO agents:**
   - **Casting Director**: Uses role_matcher and fetch_actor_database
   - **Schedule Coordinator**: Uses actor_availability_checker

4. **Define tasks:**
   - Task 1: Find suitable actors for a lead role (give a character description)
   - Task 2: Check availability of suggested actors for specific shooting dates

5. **Add observability:**
   - Use the CrewObserver to track execution
   - Export logs at the end

### 💡 Bonus Points:
- Make your character descriptions creative and unique
- Add personality to your agents' backstories
- Include error handling in your tools
- Create a visualization of the observability data

### Sample Character (feel free to create your own!):
```
Character: Captain Zara Quinn
Description: A 35-year-old space captain with a mysterious past, known for her
quick wit and exceptional piloting skills. Must convey both strength and vulnerability.
```

### 📝 Your Code Goes Below:

Good luck! 🚀

In [ ]:
# YOUR CHALLENGE CODE HERE

# Step 1: Create your custom tools
# Hint: Use @tool decorator


# Step 2: Create your MCP-style tool


# Step 3: Create your agents


# Step 4: Define your tasks


# Step 5: Set up observability


# Step 6: Create and run your crew


# Step 7: Display results and metrics

## 🎉 Congratulations!

You've completed the CrewAI practice notebook! You've learned:

✅ How to create custom tools for agents  
✅ How to implement MCP-style tools for external integrations  
✅ How to configure and use Gemini 2.5 Flash model  
✅ How to create specialized agents with specific roles  
✅ How to define and orchestrate tasks  
✅ How to implement observability and monitoring  
✅ How to export and analyze execution logs  

### 🔗 Useful Resources:

- [CrewAI Documentation](https://docs.crewai.com/)
- [Google AI Studio](https://makersuite.google.com/)
- [LangChain Tools](https://python.langchain.com/docs/modules/agents/tools/)

Happy coding! 🚀🎬